In [2]:
# Import necessary modules
import sys
sys.path.append('../fraud_detection')  # Adjust this path based on your working directory
from utils.logging_util import setup_logger

# notebooks/eda_fraud_detection.ipynb

# Import the necessary functions from your project
# from model_training import prepare_data, train_logistic_regression , train_random_forest , train_gradient_boosting ,train_decision_tree,  train_mlp,train_rnn,train_cnn,train_lstm
from model_training import (prepare_data, train_logistic_regression, train_random_forest, 
                            train_gradient_boosting, train_decision_tree, )

from model_evaluation import evaluate_model




ModuleNotFoundError: No module named 'tensorflow'

In [5]:
import tensorflow as tf
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
data_path = '../data/fraud_datas.csv'
prepared_data = prepare_data(data_path, target_column='fraud_class')

NameError: name 'prepare_data' is not defined

In [3]:

# Access the prepared data
X_train = prepared_data['X_train']
X_test = prepared_data['X_test']
y_train = prepared_data['y_train']
y_test = prepared_data['y_test']



In [5]:
# Train and save the Logistic Regression model
logistic_model_path = '../models/logistic_regression_model.pkl'
logistic_results = train_logistic_regression(X_train, X_test, y_train, y_test, logistic_model_path)

# Train and save the Random Forest model
random_forest_model_path = '../models/random_forest_model.pkl'
random_forest_results = train_random_forest(X_train, X_test, y_train, y_test, random_forest_model_path)

# Display results
print(logistic_results)
print(random_forest_results)

2024-11-16 09:11:24,561 - root - INFO - Training model: LogisticRegression.
2024-11-16 09:11:24,561 - root - INFO - Training model: LogisticRegression.
2024-11-16 09:11:24,786 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-16 09:11:24,786 - root - INFO - Model saved to ../models/logistic_regression_model.pkl.
2024-11-16 09:11:24,833 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:11:24,833 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:11:24,833 - root - INFO - Training model: RandomForestClassifier.
2024-11-16 09:11:24,833 - root - INFO - Training model: RandomForestClassifier.
2024-11-16 09:12:27,618 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-16 09:12:27,618 - root - INFO - Model saved to ../models/random_forest_model.pkl.
2024-11-16 09:12:29,391 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:12:29,391 - root - I

{'model': LogisticRegression(class_weight='balanced', max_iter=2000), 'training_time': 0.20903968811035156, 'roc_auc': np.float64(0.7580767576706717), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.64      0.77     27373\n           1       0.17      0.70      0.27      2850\n\n    accuracy                           0.65     30223\n   macro avg       0.56      0.67      0.52     30223\nweighted avg       0.88      0.65      0.72     30223\n'}
{'model': RandomForestClassifier(class_weight='balanced'), 'training_time': 62.637763261795044, 'roc_auc': np.float64(0.7693871345883797), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     27373\n           1       1.00      0.54      0.70      2850\n\n    accuracy                           0.96     30223\n   macro avg       0.98      0.77      0.84     30223\nweighted avg       0.96      0.96      0.95     30223\n'}


In [8]:
# # Import necessary functions
# from fraud_detection.model_training import train_logistic_regression, train_decision_tree, train_random_forest, train_gradient_boosting


decision_tree_model_path = '../models/decision_tree_model.pkl'

gradient_boosting_model_path = '../models/gradient_boosting_model.pkl'


# Train and save the Decision Tree model
decision_tree_results = train_decision_tree(X_train, X_test, y_train, y_test, decision_tree_model_path)


# Train and save the Gradient Boosting model
gradient_boosting_results = train_gradient_boosting(X_train, X_test, y_train, y_test, gradient_boosting_model_path)

# Display results for all models

print("\nDecision Tree Model Results:")
print(decision_tree_results)

print("\nGradient Boosting Model Results:")
print(gradient_boosting_results)


2024-11-16 09:42:03,874 - root - INFO - Training model: DecisionTreeClassifier.
2024-11-16 09:42:03,874 - root - INFO - Training model: DecisionTreeClassifier.
2024-11-16 09:42:08,093 - root - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-16 09:42:08,093 - root - INFO - Model saved to ../models/decision_tree_model.pkl.
2024-11-16 09:42:08,164 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:42:08,164 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:42:08,167 - root - INFO - Training model: GradientBoostingClassifier.
2024-11-16 09:42:08,167 - root - INFO - Training model: GradientBoostingClassifier.
2024-11-16 09:43:07,796 - root - INFO - Model saved to ../models/gradient_boosting_model.pkl.
2024-11-16 09:43:07,796 - root - INFO - Model saved to ../models/gradient_boosting_model.pkl.
2024-11-16 09:43:07,925 - root - INFO - Model training and evaluation completed successfully.
2024-11-16 09:43:07,92


Decision Tree Model Results:
{'model': DecisionTreeClassifier(), 'training_time': 4.2173683643341064, 'roc_auc': np.float64(0.754724683883017), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      0.94      0.95     27373\n           1       0.51      0.57      0.54      2850\n\n    accuracy                           0.91     30223\n   macro avg       0.73      0.75      0.74     30223\nweighted avg       0.91      0.91      0.91     30223\n'}

Gradient Boosting Model Results:
{'model': GradientBoostingClassifier(), 'training_time': 59.62649416923523, 'roc_auc': np.float64(0.7743317752606775), 'report': '              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.98     27373\n           1       1.00      0.54      0.70      2850\n\n    accuracy                           0.96     30223\n   macro avg       0.98      0.77      0.84     30223\nweighted avg       0.96      0.96      0.95     30223\n'}


In [6]:
# Extract the model objects from the training results
logistic_model = logistic_results["model"]
decision_tree_model = decision_tree_results["model"]
random_forest_model = random_forest_results["model"]
gradient_boosting_model = gradient_boosting_results["model"]

# Evaluate each model
logistic_evaluation = evaluate_model(logistic_model, X_test, y_test)
decision_tree_evaluation = evaluate_model(decision_tree_model, X_test, y_test)
random_forest_evaluation = evaluate_model(random_forest_model, X_test, y_test)
gradient_boosting_evaluation = evaluate_model(gradient_boosting_model, X_test, y_test)

# Display evaluation results
print("Logistic Regression Evaluation:")
print(logistic_evaluation)

print("\nDecision Tree Evaluation:")
print(decision_tree_evaluation)

print("\nRandom Forest Evaluation:")
print(random_forest_evaluation)

print("\nGradient Boosting Evaluation:")
print(gradient_boosting_evaluation)

2024-11-16 09:14:35,246 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-16 09:14:35,246 - root - INFO - Evaluation metrics for LogisticRegression:
2024-11-16 09:14:35,248 - root - INFO - Accuracy: 0.6482480230288191
2024-11-16 09:14:35,248 - root - INFO - Accuracy: 0.6482480230288191
2024-11-16 09:14:35,249 - root - INFO - Precision: 0.16937197246536925
2024-11-16 09:14:35,249 - root - INFO - Precision: 0.16937197246536925
2024-11-16 09:14:35,250 - root - INFO - Recall: 0.6992982456140351
2024-11-16 09:14:35,250 - root - INFO - Recall: 0.6992982456140351
2024-11-16 09:14:35,253 - root - INFO - F1 Score: 0.27269617568584525
2024-11-16 09:14:35,253 - root - INFO - F1 Score: 0.27269617568584525
2024-11-16 09:14:35,255 - root - INFO - ROC AUC: 0.7580767576706717
2024-11-16 09:14:35,255 - root - INFO - ROC AUC: 0.7580767576706717
2024-11-16 09:14:35,259 - root - INFO - Confusion Matrix: 
[[17599  9774]
 [  857  1993]]
2024-11-16 09:14:35,259 - root - INFO - Confusion Matr

Logistic Regression Evaluation:
{'accuracy': 0.6482480230288191, 'precision': np.float64(0.16937197246536925), 'recall': np.float64(0.6992982456140351), 'f1': np.float64(0.27269617568584525), 'roc_auc': np.float64(0.7580767576706717), 'confusion_matrix': array([[17599,  9774],
       [  857,  1993]])}

Decision Tree Evaluation:
{'accuracy': 0.9067928398901499, 'precision': np.float64(0.5052001260636622), 'recall': np.float64(0.5624561403508772), 'f1': np.float64(0.5322928773036693), 'roc_auc': np.float64(0.7525501759000577), 'confusion_matrix': array([[25803,  1570],
       [ 1247,  1603]])}

Random Forest Evaluation:
{'accuracy': 0.956390828177216, 'precision': np.float64(0.999348109517601), 'recall': np.float64(0.5378947368421053), 'f1': np.float64(0.6993613138686131), 'roc_auc': np.float64(0.7693871345883797), 'confusion_matrix': array([[27372,     1],
       [ 1317,  1533]])}

Gradient Boosting Evaluation:
{'accuracy': 0.9564239155609966, 'precision': np.float64(1.0), 'recall': np.

In [9]:

# Define file paths for saving models
mlp_model_path = '../models/mlp_model.h5'
rnn_model_path = '../models/rnn_model.h5'
cnn_model_path = '../models/cnn_model.h5'
lstm_model_path = '../models/lstm_model.h5'

# Train and save the MLP model
print("Training MLP model...")
mlp_model, mlp_history = train_mlp(X_train, X_test, y_train, y_test, mlp_model_path)
print("\nMLP Model Training History:")
print(mlp_history)



# Train and save the LSTM model
print("\nTraining LSTM model...")
lstm_model, lstm_history = train_lstm(X_train, X_test, y_train, y_test, lstm_model_path)
print("\nLSTM Model Training History:")
print(lstm_history)

Training MLP model...


NameError: name 'train_mlp' is not defined

In [ ]:
# Train and save the CNN model
print("\nTraining CNN model...")
cnn_model, cnn_history = train_cnn(X_train, X_test, y_train, y_test, cnn_model_path)
print("\nCNN Model Training History:")
print(cnn_history)

# Train and save the RNN model
print("\nTraining RNN model...")
rnn_model, rnn_history = train_rnn(X_train, X_test, y_train, y_test, rnn_model_path)
print("\nRNN Model Training History:")
print(rnn_history)

In [ ]:
# Evaluate the MLP, RNN, and LSTM models
mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)

# Display evaluation results
print("\nMLP Model Evaluation:")
print(mlp_evaluation)

print("\nRNN Model Evaluation:")
print(rnn_evaluation)

print("\nLSTM Model Evaluation:")
print(lstm_evaluation)


In [ ]:
# Paths to the saved models
mlp_model_path = '../models/mlp_model.h5'
rnn_model_path = '../models/rnn_model.h5'
cnn_model_path = '../models/cnn_model.h5'
lstm_model_path = '../models/lstm_model.h5'

# Load the saved models
mlp_model = load_model(mlp_model_path)
rnn_model = load_model(rnn_model_path)
cnn_model = load_model(cnn_model_path)
lstm_model = load_model(lstm_model_path)

# Test the models and evaluate them
print("Evaluating MLP model...")
mlp_evaluation = evaluate_model(mlp_model, X_test, y_test)
print("MLP Model Evaluation:", mlp_evaluation)

print("\nEvaluating RNN model...")
rnn_evaluation = evaluate_model(rnn_model, X_test, y_test)
print("RNN Model Evaluation:", rnn_evaluation)

print("\nEvaluating CNN model...")
cnn_evaluation = evaluate_model(cnn_model, X_test, y_test)
print("CNN Model Evaluation:", cnn_evaluation)

print("\nEvaluating LSTM model...")
lstm_evaluation = evaluate_model(lstm_model, X_test, y_test)
print("LSTM Model Evaluation:", lstm_evaluation)